In [ ]:
#pip install scipy
#pip install seaborn
#대구 교통 사고 피해 예측 AI 경진대회 Baseline Code

#Fixed Random Seed


import os
import random
import numpy as np


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

In [ ]:
#데이터 불러오기 및 상위행 확인
import pandas as pd
from IPython.display import display

train_org = pd.read_csv('./train.csv')
test_org = pd.read_csv('./test.csv')

display(train_org.head())
display(test_org.tail())

In [ ]:
#train, test 데이터 기간 확인하기

display(f"train : {train_org.iloc[0]['사고일시']} ~ {train_org.iloc[-1]['사고일시']}")
display(f"test : {test_org.iloc[0]['사고일시']} ~ {test_org.iloc[-1]['사고일시']}")

sample_submission = pd.read_csv("./sample_submission.csv")
print(sample_submission.head())

In [ ]:
#파생 변수 생성 1 : 날짜, 시간정보 생성

train_df = train_org.copy()
test_df = test_org.copy()

time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

train_df[['연', '월', '일', '시간']] = train_org['사고일시'].str.extract(time_pattern)
train_df[['연', '월', '일', '시간']] = train_df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
train_df = train_df.drop(columns=['사고일시']) # 정보 추출이 완료된 '사고일시' 컬럼은 제거합니다

# 해당 과정을 test_x에 대해서도 반복해줍니다
test_df[['연', '월', '일', '시간']] = test_org['사고일시'].str.extract(time_pattern)
test_df[['연', '월', '일', '시간']] = test_df[['연', '월', '일', '시간']].apply(pd.to_numeric)
test_df = test_df.drop(columns=['사고일시'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

In [ ]:
#파생 변수 생성 2 : 공간(위치) 정보 생성

location_pattern = r'(\S+) (\S+) (\S+)'

train_df[['도시', '구', '동']] = train_org['시군구'].str.extract(location_pattern)
train_df = train_df.drop(columns=['시군구'])

test_df[['도시', '구', '동']] = test_org['시군구'].str.extract(location_pattern)
test_df = test_df.drop(columns=['시군구'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

In [ ]:
#파생 변수 추출 3 : 도로 형태 정보 추출
road_pattern = r'(.+) - (.+)'

train_df[['도로형태1', '도로형태2']] = train_org['도로형태'].str.extract(road_pattern)
train_df = train_df.drop(columns=['도로형태'])

test_df[['도로형태1', '도로형태2']] = test_org['도로형태'].str.extract(road_pattern)
test_df = test_df.drop(columns=['도로형태'])

display(f"columns of train_df : {train_df.columns}")
display(f"columns of test_df : {test_df.columns}")

In [ ]:
#전처리 결과 확인

display(train_df.head())
display(test_df.head())

In [ ]:
#탐색적 데이터 분석
weekday_gu_stats = train_df.groupby(['요일','구'])['ECLO'].mean().reset_index()
display(weekday_gu_stats.head())
display(weekday_gu_stats['구'].unique())

import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.rc('font', family='Malgun Gothic')
plt.text(0.3, 0.3, '한글', size=100)
plt.rc('axes', unicode_minus=False)

for gu_name in weekday_gu_stats['구'].unique():
    subset = weekday_gu_stats[weekday_gu_stats['구'] == gu_name]
    plt.plot(subset['요일'], subset['ECLO'], marker='o', label=gu_name)


plt.xticks(ticks=range(7), labels=['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '월요일'])
plt.xlabel('요일')
plt.ylabel('사고 위험도')
plt.title('대구시 행정 구역에 따른 요일별 사고 위험도')
plt.legend(title='Building Type')
plt.grid(True)
plt.show()

In [ ]:
#train, test 데이터의 독립변수, 종속 변수 분리
test_x = test_df.drop(columns=['ID']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
#범주형(Categorical) 변수, 수치형 변수로 변환하기
from sklearn.preprocessing import LabelEncoder

categorical_features = list(train_x.dtypes[train_x.dtypes == "object"].index)
# 추출된 문자열 변수 확인
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train_x[i])
    train_x[i]=le.transform(train_x[i])

    for case in np.unique(test_x[i]):
        if case not in le.classes_:
            le.classes_ = np.append(le.classes_, case)
    test_x[i]=le.transform(test_x[i])

display(train_x.head())
display(test_x.head())

In [ ]:
#모델 검증
#Model Train & Prediction

from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
model.fit(train_x, train_y)

prediction = model.predict(test_x)
prediction

#Submission 양식 확인
baseline_submission = sample_submission.copy()
baseline_submission['ECLO'] = prediction
baseline_submission


# 답안지 저장 및 제출하기

baseline_submission.to_csv('./baseline_submit.csv', index=False)